In [ ]:
%pip install torch
%pip install torchvision
%pip install matplotlib


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split


# Constants
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 50
LEARNING_RATE = 0.0001
BATCH_SIZE = 10


def get_transforms():
    return transforms.Compose(
        [
            transforms.Pad(4),
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32),
            transforms.ToTensor(),
        ]
    )


def get_dataloaders():
    transform = get_transforms()

    train_dataset = torchvision.datasets.CIFAR10(
        root="./data", train=True, transform=transform, download=True
    )
    test_dataset = torchvision.datasets.CIFAR10(
        root="./data", train=False, transform=transforms.ToTensor()
    )

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False
    )

    return train_loader, test_loader


def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(
        in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False
    )


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels),
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


NUM_CLIENTS = 4


def split_dataset(dataset, num_clients):
    client_data_size = len(dataset) // num_clients
    sizes = [client_data_size] * num_clients
    remainder = len(dataset) % num_clients
    for i in range(remainder):
        sizes[i] += 1
    client_datasets = random_split(dataset, sizes)
    return client_datasets


def average_weights(models):
    """Average model weights from the clients."""
    averaged_weights = {}
    for key in models[0].state_dict().keys():
        averaged_weights[key] = sum(
            [model.state_dict()[key] for model in models]
        ) / len(models)
    return averaged_weights


def train_and_evaluate_federated(
    models, train_loaders, test_loader, optimizer, criterion
):
    accuracy_list = []
    total_step = len(train_loaders[0])
    curr_lr = LEARNING_RATE

    for epoch in range(NUM_EPOCHS):
        for client_id in range(NUM_CLIENTS):
            for i, (images, labels) in enumerate(train_loaders[client_id]):
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = models[client_id](images)
                loss = criterion(outputs, labels)
                optimizer[client_id].zero_grad()
                loss.backward()
                optimizer[client_id].step()

        # Average the model weights after every epoch
        averaged_weights = average_weights(models)
        for model in models:
            model.load_state_dict(averaged_weights)

        if (epoch + 1) % 20 == 0:
            curr_lr /= 3
            for opt in optimizer:
                update_lr(opt, curr_lr)

        # Evaluate using one of the models, as they have the same averaged weights
        model = models[0]
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(
            f"Epoch [{epoch+1}/{NUM_EPOCHS}] Accuracy of the federated model on the test images: {100 * correct / total} %"
        )
        accuracy_list.append(100 * correct / total)

    return accuracy_list


if __name__ == "__main__":
    train_loader, test_loader = get_dataloaders()
    subsets = split_dataset(train_loader.dataset, NUM_CLIENTS)
    train_loaders = [
        torch.utils.data.DataLoader(subset, batch_size=BATCH_SIZE, shuffle=True)
        for subset in subsets
    ]

    models = [ResNet(ResidualBlock, [2, 2, 2]).to(DEVICE) for _ in range(NUM_CLIENTS)]
    criterion = nn.CrossEntropyLoss()
    optimizers = [
        torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) for model in models
    ]

    acc = train_and_evaluate_federated(
        models, train_loaders, test_loader, optimizers, criterion
    )

    # Plot the accuracy trend after each epoch for federated learning
    plt.figure(figsize=(10, 6))
    plt.plot(acc, "-o", label="Federated Learning")
    plt.title("Project Malmantile FedAvg Trend")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()
    plt.savefig("fedavg.png")
